In [ ]:
!wget https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-G.json

--2021-10-31 11:23:24--  https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-G.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28516499 (27M) [text/plain]
Saving to: ‘toy-ppi-G.json’

toy-ppi-G.json      100%[===================>]  27.20M   145MB/s    in 0.2s    

2021-10-31 11:23:24 (145 MB/s) - ‘toy-ppi-G.json’ saved [28516499/28516499]



In [ ]:
!wget https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-class_map.json
!wget https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-walks.txt
!wget https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-id_map.json
!wget https://github.com/williamleif/GraphSAGE/blob/a0fdef95dca7b456dab01cb35034717c8b6dd017/example_data/toy-ppi-feats.npy

--2021-10-31 11:27:51--  https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-class_map.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5507260 (5.3M) [text/plain]
Saving to: ‘toy-ppi-class_map.json’

toy-ppi-class_map.j 100%[===================>]   5.25M  --.-KB/s    in 0.08s   

2021-10-31 11:27:51 (69.5 MB/s) - ‘toy-ppi-class_map.json’ saved [5507260/5507260]

--2021-10-31 11:27:52--  https://raw.githubusercontent.com/williamleif/GraphSAGE/master/example_data/toy-ppi-walks.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [22]:
import pandas as pd

In [21]:
import json
import numpy as np
from tqdm import tqdm

In [48]:
!wget https://raw.githubusercontent.com/benedekrozemberczki/MUSAE/master/input/features/git.json
!wget https://raw.githubusercontent.com/benedekrozemberczki/MUSAE/master/input/target/git_target.csv
!wget https://raw.githubusercontent.com/benedekrozemberczki/MUSAE/master/input/attenuated_edges/git_edges.csv

--2021-11-02 05:36:09--  https://raw.githubusercontent.com/benedekrozemberczki/MUSAE/master/input/features/git.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4380176 (4.2M) [text/plain]
Saving to: ‘git.json.1’

git.json.1          100%[===================>]   4.18M  --.-KB/s    in 0.03s   

2021-11-02 05:36:09 (122 MB/s) - ‘git.json.1’ saved [4380176/4380176]

--2021-11-02 05:36:09--  https://raw.githubusercontent.com/benedekrozemberczki/MUSAE/master/input/target/git_target.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [49]:
edges = pd.read_csv('git_edges.csv')
edges.columns = ['source', 'target'] # renaming for StellarGraph compatibility

with open('git.json') as json_data:
    features = json.load(json_data)
    
max_feature = np.max([v for v_list in features.values() for v in v_list])
features_matrix = np.zeros(shape = (len(list(features.keys())), max_feature+1))

i = 0
for k, vs in tqdm(features.items()):
    for v in vs:
        features_matrix[i, v] = 1
    i+=1
    
node_features = pd.DataFrame(features_matrix, index = features.keys())

# Read in targets
targets = pd.read_csv('git_target.csv')
targets.index = targets.id.astype(str)
targets = targets.loc[features.keys(), :]


100%|██████████| 37700/37700 [00:00<00:00, 146097.09it/s]


In [51]:
labels_sampled = targets['ml_target'].sample(frac=0.8, replace=False, random_state=101)

# G = sg.StellarGraph(node_features, edges.astype(str))
# G_sampled = G.subgraph(labels_sampled.index)

# print('# nodes in full graph:', len(G.nodes()))
# print('# nodes in sampled graph:', len(G_sampled.nodes()))

from sklearn import model_selection
train_labels, test_labels = model_selection.train_test_split(
    labels_sampled,
    train_size=0.05,
    random_state=42,
)

# 20% of test for validation
val_labels, test_labels = model_selection.train_test_split(
    test_labels, train_size=0.2, random_state=42,
)

In [140]:
import os.path as osp
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

dataset = 'Cora'
path = osp.join('data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]
data = data

In [142]:
edges = pd.DataFrame(data.edge_index.numpy().T)
edges.columns = ['source', 'target']

node_features = pd.DataFrame(data.x.numpy())

targets = pd.DataFrame(data.y.numpy())
targets = targets.squeeze(1)

from sklearn import model_selection

train_subjects, test_subjects =  model_selection.train_test_split(
    targets, train_size=0.1, test_size=None, stratify= targets
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)

In [146]:
def depth_sampler(n, n_sizes):
    node_lists = []
    
    # First layer
    input1 = np.array(node_features.iloc[n]).reshape(-1, 1433)
    # get all neighbours
    neighbours = edges[edges['source'] == n]['target']
  
    # randomly choose neighbours
    if len(neighbours) > 0:
        neighbours_chosen = np.random.choice(neighbours, size=n_sizes[0])
    else:
        neighbours_chosen = np.full(n_sizes[0], -1)
    input2 = np.array(node_features.iloc[neighbours_chosen])
    
    node_lists.append(list(neighbours_chosen))
    
    # Second Layer
    second_layer_list = []
    for node in neighbours_chosen:
        # get all neighbours
        if node != -1:
            neighbours = edges[edges['source'] == node]['target']
        else:
            neighbours = []
        # randomly choose neighbours
        if len(neighbours) > 0:
            neighbours_chosen = list(np.random.choice(neighbours, size=n_sizes[1]))
        else:
            neighbours_chosen = list(np.full(n_sizes[1], -1))
        
        second_layer_list += neighbours_chosen

    input3 = np.array(node_features.iloc[second_layer_list])
    
    node_lists.append(second_layer_list)
    
    return [input1, input2, input3]



In [53]:
import tensorflow as tf

In [54]:
from tensorflow.keras import Model, optimizers, metrics, losses

In [147]:
#batch
# target 1, 1433 - neigh - 10, 1433 neigh's niegh 100, 1433

#keras aggregation layer meanpooling aggregator
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
class graphSSSS(tf.keras.Model):
  def __init__(self):
    super(graphSSSS, self).__init__()

    self.dense1 = Dense(16,activation='relu') #(1, 16)
    self.dropout1 = Dropout(.2)

    self.neih = Dense(32,  activation='relu') #(10, 32)

    #aggre

    self.output2 = Dense(16, activation='relu') #(1, 16)
    self.dropout2 = Dropout(.2)
    #concat

    #2hop
    self.dense3 = Dense(16, activation='relu')  #(10, 1433) -> (2, 16)
    self.dropout3 = Dropout(.2)
    self.dense4 = Dense(32,input_shape=(-1, 10,1433) ,activation='relu' )         #neinei(10, 100, 1433) -> (10, 100, 32)
    self.dropout4 = Dropout(.2)
    self.dense5 = Dense(16, activation='relu') #2,16

    #
    self.dense6 = Dense(16, activation='relu') #
    self.dense7 = Dense(32, activation='relu')
    self.dropout5= Dropout(.2)
    self.dense8 = Dense(16,activation='relu')
    #
    self.flatten =  tf.keras.layers.Flatten()
    self.outputgra = Dense(7, activation = 'softmax') #node subjects

     


  def __call__(self, x ):
    #print("x")
    tar, nei, nienie = x #(1, 4005) (10, 4005) (100, 4005)

    output1 = self.dense1(tar) #(1, 4005) -> (1, 16)
    output1 = self.dropout1(output1)
    #print('output1 shape:', output1.shape)

    neih = self.neih(nei) #(10, 32)
    neih = self.dropout2(neih)
    #print('nieh shape:', neih.shape)


    agg = np.expand_dims(np.mean(neih, axis = 0), axis=0) #(1,32)
    #print('agg shape:', agg.shape)

    output2 = self.output2(agg) #(1, 16)
    #print('output2 shape:', output2.shape)

    #concat
    layer1 = tf.concat([output1, output2], axis = 1) #(1, 16)
    #print('layer1 shape:', layer1.shape) #1, 32

    #hop2
    output3 = self.dense3(nei)
    output3 = self.dropout3(output3)
    #print('output3 shape:', output3.shape) #(10, 4005) -> (10, 16)

    nienie = nienie.reshape(-1, 10, 1433)
    interoutput = self.dense4(nienie)
    interoutput = self.dropout4(interoutput)
    #print('interoutput shape:', interoutput.shape) #(10, 10, 4005) -> (10, 10, 32)

    agg2 = np.mean(interoutput, axis = 1) #
    #print('agg2 shape:', agg2.shape) #(10, 32)

    output4 = self.dense5(agg2)
    #print('output4 shape:', output4.shape) #(10, 16)

    layer2 = tf.concat([output3, output4], axis = 1) 
    #print('layer2 shape:', layer2.shape) #(10, 32)

    #layer1 layer2
    hop1 = self.dense6(layer1) #(1, 32) -> (1,16)
    #print('hop1 shape:', hop1.shape)

    hop2= self.dense7(layer2) #(10, 32)
    #print('hop2 shape:', hop2.shape)

    hop2 = np.expand_dims(np.mean(hop2, axis=0), axis=0) #(1, 32)
    hop2 = self.dropout5(hop2)
    #print('hop2 shape:', hop2.shape)

    hop2 = self.dense8(hop2) #(1, 16)
    #print('hop2 shape:', hop2.shape)

    l2_output =tf.concat([hop1, hop2], axis=1) # (1,16) + (1,16) = ( 1, 32) 
    #print('(Final) output shape:', l2_output.shape)
    return self.outputgra(l2_output) #0, 1

In [148]:
model = graphSSSS()

In [151]:
model = graphSSSS()
inputs = depth_sampler(54, [10, 10])
for i in range(len(inputs)):
  inputs[i]= inputs[i].astype("float32")
tar, nei, nienie = inputs
model(inputs)

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[0.14184342, 0.14316982, 0.14250474, 0.14270462, 0.14294863,
        0.1441166 , 0.14271213]], dtype=float32)>

In [153]:
loss_object = tf.keras.losses.categorical_crossentropy #SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.005)

#train_loss = tf.keras.metrics.AUC(num_thresholds=200, curve='ROC')
train_accuracy =tf.keras.metrics.AUC(num_thresholds=200, curve='ROC', name ="train_accuracy")


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [168]:
#@tf.function
def train_step(model, inputs, y, loss_object, optimizers, train_accuracy):
       
        with tf.GradientTape() as tape:
          predictions = model(inputs)
          loss = loss_object(y, predictions)
        gradients = tape.gradient(loss, model.trainable_variables) #,unconnected_gradients=tf.UnconnectedGradients.ZERO)

        optimizer.apply_gradients(
            (grad, var)
            for (grad, var) in zip(gradients, model.trainable_variables)
            if grad is not None)
        #train_loss(loss)
        train_accuracy(y, predictions)

In [172]:
for epoch in range(2):
    for x, y in tqdm(zip(train_subjects.index, train_targets)):
       inputs = depth_sampler(int(x), [10, 10])
       y = np.array(y).reshape( 1,-1).astype('float32')
       train_step(model, inputs, y, loss_object, optimizer, train_accuracy)
    print(f'Epoch {epoch + 1}, Accuracy:{train_accuracy.result()*100}')

270it [00:08, 32.35it/s]


Epoch 1, Accuracy:61.78498077392578


270it [00:08, 32.43it/s]

Epoch 2, Accuracy:65.08010864257812


In [62]:
!pip install stellargraph

     |████████████████████████████████| 435 kB 15.0 MB/s 


## Stellagraph

In [1]:
import torch
torch.__version__

'1.9.0+cu111'

In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.9.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 10.4 MB 14.8 MB/s 
     |████████████████████████████████| 3.7 MB 41.1 MB/s 
     |████████████████████████████████| 1.6 MB 35.7 MB/s 
     |████████████████████████████████| 870 kB 37.4 MB/s 


In [128]:
import networkx as nx
import pandas as pd
import os

import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [129]:
import os.path as osp
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

dataset = 'Cora'
path = osp.join('data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]
data = data

In [130]:
edges = pd.DataFrame(data.edge_index.numpy().T)
edges.columns = ['source', 'target']

In [131]:
node_features = pd.DataFrame(data.x.numpy())

In [132]:
targets = pd.DataFrame(data.y.numpy())

In [133]:
targets = targets.squeeze(1)

In [134]:
G = sg.StellarGraph(node_features, edges)

In [135]:
train_subjects, test_subjects =  model_selection.train_test_split(
    targets, train_size=0.1, test_size=None, stratify= targets
)
#one hot
target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)

In [25]:
edges_path = 'git_edges.csv'
targets_path = 'git_target.csv'
features_path = 'git.json'

# Read in edges
edges = pd.read_csv(edges_path)
edges.columns = ['source', 'target'] # renaming for StellarGraph compatibility

with open(features_path) as json_data:
    features = json.load(json_data)
    
max_feature = np.max([v for v_list in features.values() for v in v_list])
features_matrix = np.zeros(shape = (len(list(features.keys())), max_feature+1))

i = 0
for k, vs in tqdm(features.items()):
    for v in vs:
        features_matrix[i, v] = 1
    i+=1
    
node_features = pd.DataFrame(features_matrix, index = features.keys())

# Read in targets
targets = pd.read_csv(targets_path)
targets.index = targets.id.astype(str)
targets = targets.loc[features.keys(), :]

# Put the nodes, edges, and features into stellargraph structure
G = sg.StellarGraph(node_features, edges.astype(str))

100%|██████████| 37700/37700 [00:00<00:00, 139565.18it/s]


In [18]:
labels_sampled = targets['ml_target'].sample(frac=0.8, replace=False, random_state=101)
G_sampled = G.subgraph(labels_sampled.index)

print('# nodes in full graph:', len(G.nodes()))
print('# nodes in sampled graph:', len(G_sampled.nodes()))
# 5% train nodes
train_labels, test_labels = model_selection.train_test_split(
    labels_sampled,
    train_size=0.05,
    random_state=42,
)

# 20% of test for validation
val_labels, test_labels = model_selection.train_test_split(
    test_labels, train_size=0.2, random_state=42,
)

KeyError: ignored

In [136]:
# number of nodes per batch
batch_size = 50

# number of neighbours per layer
num_samples = [10, 5]

# generator
#generator = GraphSAGENodeGenerator(G_sampled, batch_size, num_samples)
generator = GraphSAGENodeGenerator(G, batch_size, num_samples)

# Generators for all the data sets
# train_gen = generator.flow(train_labels.index, train_labels, shuffle=True)
# val_gen = generator.flow(val_labels.index, val_labels)
# test_gen = generator.flow(test_labels.index, test_labels)

train_gen = generator.flow(train_subjects.index, train_targets, shuffle=True)


In [137]:
graphsage_model = GraphSAGE(
    layer_sizes=[32, 32], 
    generator=generator,
    #aggregator=MeanPoolingAggregator,
    bias=True, 
    dropout=0.5,
)

# get input and output tensors
x_inp, x_out = graphsage_model.in_out_tensors()
# pass the output tensor through the classification layer
prediction = layers.Dense(7, activation="softmax")(x_out) #sigmoid

# build and compile
from tensorflow.keras import Model

model = Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=optimizers.Adam(lr=0.005),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.AUC(num_thresholds=200, curve='ROC'), 'acc'],
)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 10, 1433)]   0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 50, 1433)]   0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 1, 1433)]    0                                            
__________________________________________________________________________________________________
reshape_20 (Reshape)            (None, 1, 10, 1433)  0           input_17[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [138]:
test_gen = generator.flow(test_subjects.index, test_targets)

In [139]:
history = model.fit(
    train_gen, 
    epochs=2, 
    validation_data=test_gen, 
    verbose=2, 
    shuffle=False)

Epoch 1/2
6/6 - 5s - loss: 1.8542 - auc_4: 0.6410 - acc: 0.2778 - val_loss: 1.7724 - val_auc_4: 0.7406 - val_acc: 0.3023
Epoch 2/2
6/6 - 2s - loss: 1.7319 - auc_4: 0.7647 - acc: 0.3000 - val_loss: 1.6823 - val_auc_4: 0.7821 - val_acc: 0.3093


## GatedGNN
# https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial9/RecGNN_tutorial.ipynb

In [202]:
import os.path as osp
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch.nn.functional as F

dataset = 'Cora'
path = osp.join('data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [203]:
from torch_geometric.nn.conv import MessagePassing
from torch.nn import Parameter as Param
from torch import Tensor 
from torch_geometric.nn.inits import uniform
import torch.nn as nn
from torch_geometric.data import DataLoader

In [204]:
class MLP(nn.Module):
    def __init__(self, input_dim, hid_dims, out_dim):
        super(MLP, self).__init__()

        self.mlp = nn.Sequential()
        dims = [input_dim] + hid_dims + [out_dim]
        for i in range(len(dims)-1):
            self.mlp.add_module('lay_{}'.format(i),nn.Linear(in_features=dims[i], out_features=dims[i+1]))
            if i+2 < len(dims):
                self.mlp.add_module('act_{}'.format(i), nn.Tanh())
    def reset_parameters(self):
        for i, l in enumerate(self.mlp):
            if type(l) == nn.Linear:
                nn.init.xavier_normal_(l.weight)

    def forward(self, x):
        return self.mlp(x)

In [205]:
class GatedGraphConv(MessagePassing):
    
    def __init__(self, out_channels, num_layers, aggr = 'add',
                 bias = True, **kwargs):
        super(GatedGraphConv, self).__init__(aggr=aggr, **kwargs)

        self.out_channels = out_channels
        self.num_layers = num_layers

        self.weight = Param(Tensor(num_layers, out_channels, out_channels))
        self.rnn = torch.nn.GRUCell(out_channels, out_channels, bias=bias) 

        self.reset_parameters()

    def reset_parameters(self):
        uniform(self.out_channels, self.weight)
        self.rnn.reset_parameters()

    def forward(self, data):
        """"""
        x = data.x
        edge_index = data.edge_index
        edge_weight = data.edge_attr
       
        zero = x.new_zeros(x.size(0), self.out_channels - x.size(-1))
        x = torch.cat([x, zero], dim=1)

        for i in range(self.num_layers):
            m = torch.matmul(x, self.weight[i])
            m = self.propagate(edge_index, x=m, edge_weight=edge_weight,
                               size=None)
            x = self.rnn(m, x) # X, H

        return x

    def message(self, x_j, edge_weight):
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)

    def __repr__(self):
        return '{}({}, num_layers={})'.format(self.__class__.__name__,
                                              self.out_channels,
                                              self.num_layers)

class GGNN(torch.nn.Module):
    def __init__(self):
        super(GGNN, self).__init__()
        
        self.conv = GatedGraphConv(1433, 3)
        self.mlp = MLP(1433, [32,32,32], dataset.num_classes)
        
    def forward(self):
        x = self.conv(data)
        x = self.mlp(x)
        return F.log_softmax(x, dim=-1)

In [206]:
model = GGNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]
test_loader = DataLoader(test_dataset)
train_loader = DataLoader(train_dataset)

def train():
    model.train()
    optimizer.zero_grad()
    loss_fn(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [208]:
for epoch in range(1, 10):
    train()
    accs = test()
    train_acc = accs[0]
    val_acc = accs[1]
    test_acc = accs[2]
    print('Epoch: {:03d}, Train Acc: {:.5f}, '
          'Val Acc: {:.5f}, Test Acc: {:.5f}'.format(epoch, train_acc,
                                                       val_acc, test_acc))

Epoch: 001, Train Acc: 0.22143, Val Acc: 0.21600, Test Acc: 0.20000
Epoch: 002, Train Acc: 0.24286, Val Acc: 0.23400, Test Acc: 0.21200
Epoch: 003, Train Acc: 0.31429, Val Acc: 0.25600, Test Acc: 0.24100
Epoch: 004, Train Acc: 0.43571, Val Acc: 0.34400, Test Acc: 0.35100
Epoch: 005, Train Acc: 0.50000, Val Acc: 0.48400, Test Acc: 0.49200
Epoch: 006, Train Acc: 0.46429, Val Acc: 0.43200, Test Acc: 0.44800
Epoch: 007, Train Acc: 0.45714, Val Acc: 0.51600, Test Acc: 0.51500
Epoch: 008, Train Acc: 0.56429, Val Acc: 0.48000, Test Acc: 0.48700
Epoch: 009, Train Acc: 0.53571, Val Acc: 0.44200, Test Acc: 0.46700


In [390]:
#GatedGraphConv(1433, 3)
import torch

from torch.nn import Parameter as Param
from torch import Tensor 

weight = Param(Tensor(3, 1433, 1433))
x = data.x #x.size(-1) 1433
zero =  x.new_zeros(x.size(0), 1433 - x.size(-1)) 
x = torch.cat([x, zero], dim=1)#.shape

w1 = weight[0] #(1433, 1433)
m = torch.matmul(x, w1)
m.shape 


#Adj, X , W
#(2708, 1433)
torch.nn.GRUCell(out_channels, out_channels, bias=True)

x  = rnn(m, x) 



# Graph Attention Network

## https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial3/Tutorial3.ipynb

In [209]:
#GAT
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [12]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features))) #xavier paramiter inizializator
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes,in_features) 


# linear transformation
h = torch.mm(input, W) #input (3,5) W (5, 2) -> #(3,2)
N = h.size()[0]

print(h)#(3,2)

tensor([[-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256]], grad_fn=<MmBackward>)


In [9]:
a =  nn.Parameter(torch.zeros(size=(2*out_features, 1))) #i input 2+ j input 2 / 4, 1
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a) #(4,1)

Parameter containing:
tensor([[ 0.9236],
        [ 0.3101],
        [-1.1088],
        [-1.1281]], requires_grad=True)


In [49]:
a

Parameter containing:
tensor([[ 0.9236],
        [ 0.3101],
        [-1.1088],
        [-1.1281]], requires_grad=True)

In [45]:
h

tensor([[-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256]], grad_fn=<MmBackward>)

In [46]:
a_input = torch.cat([h.repeat(1,N).view(N*N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2*out_features) #(18, 2) -> (3, 3, 4)

In [47]:
a_input

tensor([[[-0.8810, -0.0695, -0.8810, -0.0695],
         [-0.8810, -0.0695, -1.4563, -0.1818],
         [-0.8810, -0.0695, -1.1797,  0.4256]],

        [[-1.4563, -0.1818, -0.8810, -0.0695],
         [-1.4563, -0.1818, -1.4563, -0.1818],
         [-1.4563, -0.1818, -1.1797,  0.4256]],

        [[-1.1797,  0.4256, -0.8810, -0.0695],
         [-1.1797,  0.4256, -1.4563, -0.1818],
         [-1.1797,  0.4256, -1.1797,  0.4256]]], grad_fn=<ViewBackward>)

In [15]:
h.repeat(1, N).view(N*N, -1)
#N 3 (3, 6) -> (9, 2)

tensor([[-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256],
        [-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256],
        [-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256]], grad_fn=<RepeatBackward>)

In [16]:
h.repeat(N, 1) #(9, 2) 

tensor([[-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256],
        [-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256],
        [-0.8810, -0.0695],
        [-1.4563, -0.1818],
        [-1.1797,  0.4256]], grad_fn=<RepeatBackward>)

In [51]:
a_input.shape

torch.Size([3, 3, 4])

In [92]:
a_input #(3, 3, 4)

tensor([[[-0.8810, -0.0695, -0.8810, -0.0695],
         [-0.8810, -0.0695, -1.4563, -0.1818],
         [-0.8810, -0.0695, -1.1797,  0.4256]],

        [[-1.4563, -0.1818, -0.8810, -0.0695],
         [-1.4563, -0.1818, -1.4563, -0.1818],
         [-1.4563, -0.1818, -1.1797,  0.4256]],

        [[-1.1797,  0.4256, -0.8810, -0.0695],
         [-1.1797,  0.4256, -1.4563, -0.1818],
         [-1.1797,  0.4256, -1.1797,  0.4256]]], grad_fn=<ViewBackward>)

In [81]:
torch.matmul(a_input, a).squeeze(2)

tensor([[[-1.9009, -1.9009, -1.9009],
         [-2.5886, -2.5886, -2.5886],
         [-1.7046, -1.7046, -1.7046]],

        [[-2.5886, -2.5886, -2.5886],
         [-3.2762, -3.2762, -3.2762],
         [-2.3922, -2.3922, -2.3922]],

        [[-1.7046, -1.7046, -1.7046],
         [-2.3922, -2.3922, -2.3922],
         [-1.5083, -1.5083, -1.5083]]], grad_fn=<SqueezeBackward1>)

In [68]:
#tf.transpose(a_input, perm = [0,2,1])
#a_input.permutate(0, 2, 1)
a = torch.transpose(a_input, 2, 1)

In [40]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2)) #(3, 3, 1) -squeeze-> (3, 3)
e

tensor([[ 0.2200,  0.9846, -0.0015],
        [-0.0692,  0.4184, -0.1147],
        [ 0.0976,  0.8622, -0.0259]], grad_fn=<LeakyReluBackward0>)

In [37]:
adj = torch.randint(2, (3, 3))
zero_vec = -9e15*torch.ones_like(e)
print(zero_vec)

tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])


In [38]:
attention = torch.where(adj>0, e, zero_vec)
attention

tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-6.9242e-02,  4.1844e-01, -9.0000e+15],
        [ 9.7600e-02,  8.6225e-01, -9.0000e+15]], grad_fn=<SWhereBackward>)

In [44]:
attention = F.softmax(attention, dim = 1)
h_prime = torch.matmul(attention, h)

# Trainable Parameter ; W(features_n, out(hidden feat)_n), a(hfeatures_n*2, 1)
# a = neighbor attention, why anisotropic???

In [210]:
import torch.nn as nn

In [277]:
class GATLayer(nn.Module):
  def __init__(self, in_features, out_features, dropout, alpha, concat = True):
    super(GATLayer, self).__init__()
    self.dropout = dropout #0.6
    self.in_features = in_features
    self.out_features = out_features #hidden
    self.alpha = alpha # 0.2
    self.concat = concat

    #trainable W, a
    self.W = nn.Parameter(torch.zeros(size = (in_features, out_features)))
    nn.init.xavier_uniform_(self.W.data, gain=1.414)
    self.a = nn.Parameter(torch.zeros(size = (2*out_features, 1)))
    nn.init.xavier_uniform_(self.a.data, gain=1.414)

    self.leakyrelu = nn.LeakyReLU(self.alpha) #0.2

  def forward(self, input, adj): #input (nod_num, x_feature), adj
    h = torch.mm(input, self.W) # (nod, xw_feat)
    N = h.size()[0]

    #attention mechanism nod*nod - each node
    a_input = torch.cat([h.repeat(1, N).view(N*N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
    e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2)) # a :(out_feat *2 , 1) | (node, node, 1) -> node * node
    #a_input * a = (node, node, 1) - squez - (node, node)

    #mask
    zero_vec = -9e15*torch.ones_like(e)
    attention = torch.where(adj >0, e, zero_vec)

    attention = F.softmax(attention, dim=1)
    attnetion = F.dropout(attention, self.dropout, training = self.training)

    h_prime = torch.matmul(attention, h) #Value

    if self.concat:
            return F.elu(h_prime).squeeze(0)
    else:
        return h_prime.squeeze(0)

In [220]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

name_data = 'Cora'
dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)

Number of Classes in Cora: 7
Number of Node Features in Cora: 1433


In [243]:
import torch_geometric

In [245]:
adj = torch_geometric.utils.to_dense_adj(data.edge_index)

In [289]:
gat = GAT()
gat(data).shape

torch.Size([2708, 7])

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        self.conv1 =  GATLayer(dataset.num_features, self.hid, dropout = 0.6, alpha = 0.2)
        self.conv2 = GATLayer(self.hid, dataset.num_classes, concat = False , dropout = 0.6, alpha = 0.2)
        
        # self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        # self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
        #                      heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_index = torch_geometric.utils.to_dense_adj(data.edge_index)
                
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)
    
    
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"

model = GAT().to(device)
data = dataset[0].to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    
    
    print(loss)
    
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

tensor(1.9452, grad_fn=<NllLossBackward>)
tensor(1.9439, grad_fn=<NllLossBackward>)
tensor(1.9390, grad_fn=<NllLossBackward>)
tensor(1.9349, grad_fn=<NllLossBackward>)
tensor(1.9286, grad_fn=<NllLossBackward>)
tensor(1.9306, grad_fn=<NllLossBackward>)
tensor(1.9206, grad_fn=<NllLossBackward>)
tensor(1.9195, grad_fn=<NllLossBackward>)
tensor(1.9148, grad_fn=<NllLossBackward>)
